<a href="https://colab.research.google.com/github/BrooklynZhang/Personal/blob/master/nlp_extra_credit_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import json
import os
import re

batch_size = 64
buffer_size = 1024
save_path = "model/"
encoder_file = save_path + "encoder"


def load_data(jsonfile, labels):
    data = []
    result = []
    with open(jsonfile, "r") as data_file:
        dataset = json.load(data_file)
        for id, post in dataset.items():
            doc = post["body"]
            labellist = []
            labellist.append(int(post["emotion"][labels]))

            data.append(doc)
            result.append(labellist)
    result = np.asarray(result, dtype=np.int64)
    dataset = tf.data.Dataset.from_tensor_slices((data, result))
    return dataset


def create_model(labels, encoder=None):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(encoder.vocab_size, 64),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    return model

def init_model(label):
    model = create_model(label, encoder=encoder)
    encoder.save_to_file(encoder_file)
    tf.keras.models.save_model(model, save_path + str(label))


def main(label):
    if not os.path.exists(save_path + str(label)):
        os.makedirs(save_path + str(label))

    post_dataset = load_data("nlp_train.json", label)

    print("Loading model from {}".format(save_path + str(label)))
    model = tf.keras.models.load_model(save_path + str(label))
    encoder = tfds.features.text.TokenTextEncoder.load_from_file(encoder_file)
    

    def encode_map_fn(text, label): #encode the text to integers
        def encode(text_tensor, label):
            encoded_text = encoder.encode(text_tensor.numpy())
            return encoded_text, label

        encoded_text, label = tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

        encoded_text.set_shape([None])
        label.set_shape([None])
        return encoded_text, label

    dataset_encoded = post_dataset.map(encode_map_fn)

    x_test = dataset_encoded.take(100)
    x_test = x_test.padded_batch(batch_size, padded_shapes=([None],[None]))
    
    x_train = dataset_encoded.skip(100).shuffle(buffer_size)
    x_train = x_train.padded_batch(batch_size, padded_shapes=([None],[None]))

    model.compile(loss=tf.keras.losses.MeanSquaredError(), metrics=["accuracy"], optimizer=tf.keras.optimizers.Adam(1e-4))

    for i in range(5):
        history = model.fit(x_train, epochs=10, batch_size=batch_size, validation_data=x_test)
        tf.keras.models.save_model(model, save_path + str(label))



if __name__ == "__main__":
    total_labels = ["anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust", "neutral"]

    '''post_dataset = load_data("nlp_train.json", "anger")

    tokenizer = tfds.features.text.Tokenizer()
    vocabulary_set = set()
    for text_tensor, _ in post_dataset:
        tokens = tokenizer.tokenize(text_tensor.numpy())
        vocabulary_set.update(tokens)

    encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)
    for l in total_labels:
        init_model(l)'''
    for l in total_labels:
        main(l)

    
  

Loading model from model/anger
Epoch 1/10
22/22 [==============================] - 5s 212ms/step - loss: 0.3607 - accuracy: 0.6339 - val_loss: 0.3064 - val_accuracy: 0.6600
Epoch 2/10
22/22 [==============================] - 4s 178ms/step - loss: 0.3044 - accuracy: 0.6339 - val_loss: 0.2598 - val_accuracy: 0.6600
Epoch 3/10
22/22 [==============================] - 4s 183ms/step - loss: 0.2510 - accuracy: 0.6339 - val_loss: 0.2236 - val_accuracy: 0.6600
Epoch 4/10
22/22 [==============================] - 4s 181ms/step - loss: 0.2272 - accuracy: 0.6339 - val_loss: 0.2227 - val_accuracy: 0.6600
Epoch 5/10
22/22 [==============================] - 4s 179ms/step - loss: 0.2203 - accuracy: 0.6339 - val_loss: 0.2193 - val_accuracy: 0.6600
Epoch 6/10
22/22 [==============================] - 4s 185ms/step - loss: 0.2083 - accuracy: 0.6612 - val_loss: 0.2156 - val_accuracy: 0.6700
Epoch 7/10
22/22 [==============================] - 4s 184ms/step - loss: 0.1807 - accuracy: 0.7954 - val_loss: 0.197